# 画像による物体検出

Matplotlib と Pillow のインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q matplotlib Pillow

In [ ]:
%run setup_demo_env.py

In [ ]:
import io

import grpc
import kachaka_api_pb2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
from kachaka_api_pb2_grpc import KachakaApiStub
from PIL import Image

stub = KachakaApiStub(grpc.aio.insecure_channel(kachaka_api_server))

LABEL_NAME = ["?", "person", "shelf", "charger", "door"]
LABEL_COLOR = ["pink", "green", "blue", "cyan", "red"]

req_i = kachaka_api_pb2.GetRequest()
req_d = kachaka_api_pb2.GetRequest()
while True:
    camera_promise = stub.GetFrontCameraRosCompressedImage(req_i)
    detection_promise = stub.GetObjectDetection(req_d)
    resp_i = await camera_promise
    resp_d = await detection_promise
    req_i.metadata.cursor = resp_i.metadata.cursor
    req_d.metadata.cursor = resp_d.metadata.cursor

    clear_output(wait=True)
    fig, ax = plt.subplots(1)

    img = Image.open(io.BytesIO(resp_i.image.data))
    ax.imshow(img)

    for obj in resp_d.objects:
        x, y, w, h = obj.roi.x_offset, obj.roi.y_offset, obj.roi.width, obj.roi.height
        rect = patches.Rectangle(
            (x, y),
            w,
            h,
            linewidth=2,
            edgecolor=LABEL_COLOR[obj.label],
            facecolor="none",
        )
        ax.add_patch(rect)
        label = f"{LABEL_NAME[obj.label]}, score={obj.score:.2f}"
        plt.text(x, y + h, label, color="white", backgroundcolor="grey")

    plt.axis("off")
    plt.show()